In [11]:
import os
sys.path.append(os.path.abspath(".."))
from shopping_agent import run_shopping_agent

In [19]:
# instruction = "لطفاً دراور چهار کشو (کد D14) را برای من تهیه کنید."
# instruction = "عرض پارچه تریکو جودون 1/30 لاکرا گردباف نوریس به رنگ زرد طلایی چقدر است؟"
instruction = "کمترین قیمت در این پایه برای گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸ چقدر است؟"
# instruction = "یک لپ تاپ مناسب بهم پیشنهاد کن."
result = await run_shopping_agent(instruction)

Plan:

1. Translate the user request: The user wants to know the lowest price for the base product named "گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸" (Natural Plant Black Gold Bonsai Nargol Code 0108).

2. Task: Find the base product with the exact Persian name "گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸" in the base_products table.

3. Once the base product is found, get its random_key.

4. Use this random_key to find all related shop products (members) in the members table.

5. From these members, find the minimum price.

6. Output the minimum price.

Subtasks and tools:

- Use build_exact_query_and_execute on base_products with column persian_name = "گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸" to get the base product random_key.

- Use build_exact_query_and_execute on members with column base_random_key = found random_key to get prices.

- Find the minimum price from the returned members.

Output:

- message: The minimum price found.

- base_random_keys and member_random_keys: null (not request

In [20]:
result

AgentRunResult(output=ShoppingResponse(message='کمترین قیمت برای پایه محصول گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸ برابر با 275000 تومان است.', base_random_keys=None, member_random_keys=None))

In [21]:
dict(result.output)

{'message': 'کمترین قیمت برای پایه محصول گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸ برابر با 275000 تومان است.',
 'base_random_keys': None,
 'member_random_keys': None}

In [22]:
result.usage()

RunUsage(input_tokens=10884, cache_read_tokens=7936, output_tokens=268, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, requests=5, tool_calls=4)

In [23]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='\n\nYou are an AI Shopping Assistant for Torob. Your job is to answer user instructions by retrieving product information.\n\n\n\nYou have access to the following tools:\n\n1. build_exact_query_and_execute(table_name: str, column_name: str, variable_query: str, limit: int = 10, columns: list[str] | None = None) -> list[RealDictRow] | str:\n   Builds and executes a SQL query that searches for rows where a column exactly matches\n   the given value. You can specify which columns to return. Enforces a minimum limit of 1.\n\n2. build_like_query_and_execute(table_name: str, column_name: str, variable_query: str, limit: int = 10, columns: list[str] | None = None) -> list[RealDictRow] | str:\n   Builds and executes a SQL query that searches for rows where a column contains\n   a given substring (case-insensitive). You can specify which columns to return.\n   Enforces a minimum limit of 3.\n   \n3. generate_sql_query(instruction: str) -> tuple[str